<a href="https://colab.research.google.com/github/mounikapentukar-alt/Projects/blob/main/HealthCare_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load all three datasets and inspect their structure and missing values.
import pandas as pd
import numpy as np

#Data loading
patients = pd.read_csv("/content/patient.csv")
admissions = pd.read_csv("/content/admissions.csv")
billing = pd.read_csv("/content/billing.csv")

#schema inspection
patients.info()
admissions.info()
billing.info()

#analysis of null values
patients.isna().sum()
admissions.isna().sum()
billing.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   patient_id          38 non-null     int64 
 1   age                 38 non-null     int64 
 2   gender              38 non-null     object
 3   zip_code            38 non-null     int64 
 4   chronic_conditions  25 non-null     object
 5   insurance_type      38 non-null     object
 6   registration_date   38 non-null     object
dtypes: int64(3), object(4)
memory usage: 2.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   admission_id    50 non-null     int64 
 1   patient_id      50 non-null     int64 
 2   admission_date  50 non-null     object
 3   discharge_date  50 non-null     object
 4   department      50 non-null     object
 5   l

,0
patient_id,0
total_charge,0
insurance_covered,0
patient_paid,0
claim_status,0


In [ ]:
#What is the age distribution of patients and how does it vary by gender?
patients.groupby("gender")["age"].agg(
    min_age="min",
    max_age="max",
    avg_age="mean"
).round(1)


,min_age,max_age,avg_age
gender,,,
Female,26,86,51.1
Male,23,88,50.9
Other,38,66,52.0


In [ ]:
#How many patients have at least one chronic condition?
patients["has_chronic"] = np.where(
    patients["chronic_conditions"] == "None", 0, 1
)

patients["has_chronic"].value_counts()


,count
has_chronic,
1,38


In [ ]:
#What is the average and maximum length of stay by department?
admissions.groupby("department")["length_of_stay"].agg(
    avg_los="mean",
    max_los="max"
).round(2)

Object `department` not found.


,avg_los,max_los
department,,
Cardiology,7.12,13
Emergency,6.31,12
General Medicine,9.00,13
Neurology,9.67,14
Oncology,7.80,14
Orthopedics,6.14,11


In [ ]:
#Which admissions reference patient IDs that do not exist in the patient table?


missing_patients = admissions[
    ~admissions["patient_id"].isin(patients["patient_id"])
]

missing_patients[["admission_id", "patient_id"]]

,admission_id,patient_id
0,2323232,906572
6,2041832,976569
30,2185652,941104
33,2025990,998038
40,2431712,962493
41,2841204,907492
46,2260222,965909


In [ ]:
#Which patients never had a hospital admission?

patients_no_admission = patients[
    ~patients["patient_id"].isin(admissions["patient_id"])
]

patients_no_admission[["patient_id", "age", "gender"]]

,patient_id,age,gender
6,334053,61,Male
7,246316,31,Female
8,872246,29,Female
14,719176,23,Male
18,198246,66,Female
22,731262,64,Female
26,850800,47,Female
27,781453,55,Female
28,835392,28,Female
29,671412,47,Male


In [ ]:
#Create a combined dataset of patients and their admissions.


patient_admissions = patients.merge(
    admissions,
    on="patient_id",
    how="inner"
)

patient_admissions.head()


,patient_id,age,gender,zip_code,chronic_conditions,insurance_type,registration_date,has_chronic,admission_id,admission_date,discharge_date,department,length_of_stay,admission_type
0,770487,38,Female,60601,Heart Disease,Private,2019-12-07,1,2715198,2023-06-08,2023-06-10,Cardiology,2,Emergency
1,216739,72,Female,60601,Heart Disease,Private,2021-04-12,1,2261650,2022-07-14,2022-07-19,Emergency,5,Observation
2,126225,61,Male,49503,Asthma,Medicare,2021-09-21,1,2927082,2023-06-04,2023-06-09,Orthopedics,5,Observation
3,126225,61,Male,49503,Asthma,Medicare,2021-09-21,1,2975050,2023-03-29,2023-04-09,Emergency,11,Observation
4,126225,61,Male,49503,Asthma,Medicare,2021-09-21,1,2906604,2022-03-28,2022-04-02,Oncology,5,Emergency


In [ ]:
#Identify patients older than 65 with hospital stays longer than 7 days.


high_risk = patient_admissions[
    (patient_admissions["age"] > 65) &
    (patient_admissions["length_of_stay"] > 7)
]

high_risk[["patient_id", "age", "department", "length_of_stay"]]

,patient_id,age,department,length_of_stay
10,207473,66,General Medicine,13
24,343962,88,Cardiology,11


In [ ]:
#Which billing records show patients paying more than insurance covered?


billing_overpay = billing[
    billing["patient_paid"] > billing["insurance_covered"]
]

billing_overpay

,patient_id,total_charge,insurance_covered,patient_paid,claim_status
1,131244,8433,2180,3033,Approved
12,809570,22862,965,4853,Approved
23,331148,12043,1430,9066,Pending
44,207473,25337,8135,9453,Denied


In [ ]:
#Create a simple risk score using age and number of admissions.

admission_counts = admissions.groupby("patient_id").size()

patients["admission_count"] = patients["patient_id"].map(admission_counts).fillna(0)

patients["risk_score"] = np.where(
    (patients["age"] > 65) & (patients["admission_count"] >= 2),
    "High",
    np.where(
        (patients["age"] > 50) & (patients["admission_count"] >= 1),
        "Medium",
        "Low"
    )
)

patients[["patient_id", "age", "admission_count", "risk_score"]].head()

,patient_id,age,admission_count,risk_score
0,770487,38,1.0,Low
1,216739,72,1.0,Medium
2,126225,61,4.0,Medium
3,877572,53,2.0,Medium
4,388389,37,1.0,Low


In [ ]:
#Create a final analytical table combining patients, admissions, and billing.
final_df = (
    patients
    .merge(admissions, on="patient_id", how="left")
    .merge(billing, on="patient_id", how="left")
)

final_df.head()


,patient_id,age,gender,zip_code,chronic_conditions,insurance_type,registration_date,admission_count,risk_score,admission_id,admission_date,discharge_date,department,length_of_stay,admission_type,total_charge,insurance_covered,patient_paid,claim_status
0,770487,38,Female,60601,Heart Disease,Private,2019-12-07,1.0,Low,2715198.0,2023-06-08,2023-06-10,Cardiology,2.0,Emergency,44290.0,20979.0,1213.0,Approved
1,216739,72,Female,60601,Heart Disease,Private,2021-04-12,1.0,Medium,2261650.0,2022-07-14,2022-07-19,Emergency,5.0,Observation,NaN,NaN,NaN,NaN
2,126225,61,Male,49503,Asthma,Medicare,2021-09-21,4.0,Medium,2927082.0,2023-06-04,2023-06-09,Orthopedics,5.0,Observation,NaN,NaN,NaN,NaN
3,126225,61,Male,49503,Asthma,Medicare,2021-09-21,4.0,Medium,2975050.0,2023-03-29,2023-04-09,Emergency,11.0,Observation,NaN,NaN,NaN,NaN
4,126225,61,Male,49503,Asthma,Medicare,2021-09-21,4.0,Medium,2906604.0,2022-03-28,2022-04-02,Oncology,5.0,Emergency,NaN,NaN,NaN,NaN
